In [1]:
import torch

In [2]:
x=torch.randn(5,3)

In [4]:
x

tensor([[ 0.9586, -1.2168, -0.5338],
        [-0.9936,  0.5299, -0.0387],
        [-0.4098,  1.5394,  0.0322],
        [ 1.3615,  0.2598,  0.9574],
        [-0.7260,  0.7264,  0.3477]])

In [5]:
x.size()

torch.Size([5, 3])

In [7]:
y=x[0:]
y

tensor([[ 0.9586, -1.2168, -0.5338],
        [-0.9936,  0.5299, -0.0387],
        [-0.4098,  1.5394,  0.0322],
        [ 1.3615,  0.2598,  0.9574],
        [-0.7260,  0.7264,  0.3477]])

In [8]:
y+=1

In [10]:
x# 源tensor也被改了

tensor([[ 1.9586, -0.2168,  0.4662],
        [ 0.0064,  1.5299,  0.9613],
        [ 0.5902,  2.5394,  1.0322],
        [ 2.3615,  1.2598,  1.9574],
        [ 0.2740,  1.7264,  1.3477]])

### 改变形状
用`view()`来改变`Tensor`的形状：

In [11]:
y = x.view(15)
z = x.view(-1, 5)  # -1所指的维度可以根据其他维度的值推出来
print(x.size(), y.size(), z.size())

torch.Size([5, 3]) torch.Size([15]) torch.Size([3, 5])


**注意`view()`返回的新tensor与源tensor共享内存（其实是同一个tensor），也即更改其中的一个，另外一个也会跟着改变。(顾名思义，view仅仅是改变了对这个张量的观察角度)**

In [13]:
x+=1
print(x)
print(y)# y也+1

tensor([[3.9586, 1.7832, 2.4662],
        [2.0064, 3.5299, 2.9613],
        [2.5902, 4.5394, 3.0322],
        [4.3615, 3.2598, 3.9574],
        [2.2740, 3.7264, 3.3477]])
tensor([3.9586, 1.7832, 2.4662, 2.0064, 3.5299, 2.9613, 2.5902, 4.5394, 3.0322,
        4.3615, 3.2598, 3.9574, 2.2740, 3.7264, 3.3477])


所以如果我们想返回一个真正新的副本（即不共享内存）该怎么办呢？Pytorch还提供了一个`reshape()`可以改变形状，但是此函数并不能保证返回的是其拷贝，所以不推荐使用。推荐先用`clone`创造一个副本然后再使用`view`。[参考此处](

In [14]:
x_cp=x.clone().view(15)

In [15]:
x_cp

tensor([3.9586, 1.7832, 2.4662, 2.0064, 3.5299, 2.9613, 2.5902, 4.5394, 3.0322,
        4.3615, 3.2598, 3.9574, 2.2740, 3.7264, 3.3477])

In [16]:
x-=1
print(x)
print(x_cp)

tensor([[2.9586, 0.7832, 1.4662],
        [1.0064, 2.5299, 1.9613],
        [1.5902, 3.5394, 2.0322],
        [3.3615, 2.2598, 2.9574],
        [1.2740, 2.7264, 2.3477]])
tensor([3.9586, 1.7832, 2.4662, 2.0064, 3.5299, 2.9613, 2.5902, 4.5394, 3.0322,
        4.3615, 3.2598, 3.9574, 2.2740, 3.7264, 3.3477])


> 使用`clone`还有一个好处是会被记录在计算图中，即梯度回传到副本时也会传到源`Tensor`。

另外一个常用的函数就是`item()`, 它可以将一个标量`Tensor`转换成一个Python number：

In [17]:
x = torch.randn(1)
print(x)
print(x.item())

tensor([0.0628])
0.06281913816928864


## 2.2.3 广播机制

前面我们看到如何对两个形状相同的`Tensor`做按元素运算。当对两个形状不同的`Tensor`按元素运算时，可能会触发广播（broadcasting）机制：先适当复制元素使这两个`Tensor`形状相同后再按元素运算。例如：

In [21]:
x=torch.arange(1,3).view(1,2)
print(x)
y=torch.arange(1,4).view(3,1)
print(y)
print(x+y)

tensor([[1, 2]])
tensor([[1],
        [2],
        [3]])
tensor([[2, 3],
        [3, 4],
        [4, 5]])


由于`x`和`y`分别是1行2列和3行1列的矩阵，如果要计算`x + y`，那么`x`中第一行的2个元素被广播（复制）到了第二行和第三行，而`y`中第一列的3个元素被广播（复制）到了第二列。如此，就可以对2个3行2列的矩阵按元素相加。

## 2.2.4 运算的内存开销

前面说了，索引、`view`是不会开辟新内存的，而像`y = x + y`这样的运算是会新开内存的，然后将`y`指向新内存。为了演示这一点，我们可以使用Python自带的`id`函数：如果两个实例的ID一致，那么它们所对应的内存地址相同；反之则不同。

In [23]:
x=torch.tensor([1,2])
y=torch.tensor([3,4])
id_before=id(y)
y=y+x
print(id(y)==id_before)

False


如果想指定结果到原来的`y`的内存，我们可以使用前面介绍的索引来进行替换操作。在下面的例子中，我们把`x + y`的结果通过`[:]`写进`y`对应的内存中。

In [24]:
x = torch.tensor([1, 2])
y = torch.tensor([3, 4])
id_before = id(y)
y[:] = y + x         #  y[:]
print(id(y) == id_before) # True

True


我们还可以使用运算符全名函数中的`out`参数或者自加运算符`+=`(也即`add_()`)达到上述效果，例如`torch.add(x, y, out=y)`和`y += x`(`y.add_(x)`)。

In [25]:
x = torch.tensor([1, 2])
y = torch.tensor([3, 4])
id_before = id(y)
torch.add(x, y, out=y) # y += x, y.add_(x)
print(id(y) == id_before) # True

True


> 所以只有y=y+x会导致重新开辟内存?

### 2.2.5 `Tensor`和NumPy相互转换

我们很容易用`numpy()`和`from_numpy()`将`Tensor`和NumPy中的数组相互转换。但是需要注意的一点是：
**这两个函数所产生的的`Tensor`和NumPy中的数组共享相同的内存（所以他们之间的转换很快），改变其中一个时另一个也会改变！！！**

> 还有一个常用的将NumPy中的array转换成`Tensor`的方法就是`torch.tensor()`, 需要注意的是，此方法总是会进行数据拷贝（就会消耗更多的时间和空间），所以返回的`Tensor`和原来的数据不再共享内存。

### `Tensor`转NumPy

使用`numpy()`将`Tensor`转换成NumPy数组:

In [30]:
a = torch.ones(5)
b = a.numpy()
print(a, b)

a += 1
print(a, b)
b += 1
print(a, b)

tensor([1., 1., 1., 1., 1.]) [1. 1. 1. 1. 1.]
tensor([2., 2., 2., 2., 2.]) [2. 2. 2. 2. 2.]
tensor([3., 3., 3., 3., 3.]) [3. 3. 3. 3. 3.]


### NumPy数组转`Tensor`

使用`from_numpy()`将NumPy数组转换成`Tensor`:

In [28]:
import numpy as np
a=np.ones(5)
b=torch.from_numpy(a)
print(a,b)

a += 1
print(a, b)
b += 1
print(a, b)

[1. 1. 1. 1. 1.] tensor([1., 1., 1., 1., 1.], dtype=torch.float64)
[2. 2. 2. 2. 2.] tensor([2., 2., 2., 2., 2.], dtype=torch.float64)
[3. 3. 3. 3. 3.] tensor([3., 3., 3., 3., 3.], dtype=torch.float64)


所有在CPU上的`Tensor`（除了`CharTensor`）都支持与NumPy数组相互转换。

此外上面提到还有一个常用的方法就是直接用`torch.tensor()`将NumPy数组转换成`Tensor`，需要注意的是该方法总是会进行数据拷贝，返回的`Tensor`和原来的数据不再共享内存。

In [29]:
c = torch.tensor(a)
a += 1
print(a, c)

[4. 4. 4. 4. 4.] tensor([3., 3., 3., 3., 3.], dtype=torch.float64)


## 2.2.6 `Tensor` on GPU

用方法`to()`可以将`Tensor`在CPU和GPU（需要硬件支持）之间相互移动。

In [32]:
# 以下代码只有在PyTorch GPU版本上才会执行
if torch.cuda.is_available():
    device = torch.device("cuda")          # GPU
    y = torch.ones_like(x, device=device)  # 直接创建一个在GPU上的Tensor
    x = x.to(device)                       # 等价于 .to("cuda")
    z = x + y
    print(z)
    print(z.to("cpu", torch.double))       # to()还可以同时更改数据类型